In [1]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import random

# Initialize MediaPipe Pose and KNN
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
knn = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=3))

# Function to extract pose key points from an image
def extract_keypoints(image):
    if image is None:
        print("Warning: Empty image passed to extract_keypoints function.")
        return None
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.pose_landmarks:
        return None
    keypoints = []
    for landmark in results.pose_landmarks.landmark:
        keypoints.append((landmark.x, landmark.y))
    return np.array(keypoints).flatten()

# Function to augment images
def augment_image(image):
    rows, cols = image.shape[:2]
    angle = random.uniform(-10, 10)
    scale = random.uniform(0.9, 1.1)
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, scale)
    augmented_image = cv2.warpAffine(image, M, (cols, rows))
    return augmented_image

# Load pose images and non-pose images, then extract keypoints
pose_images = [cv2.imread(f'PoseData/{i}.png') for i in range(1, 11)]  # Replace with actual paths
non_pose_images = [cv2.imread(f'PoseData/{i}.png') for i in range(1, 11)]  # Replace with paths to non-pose images

data = []
labels = []

# Process target pose images
for img in pose_images:
    keypoints = extract_keypoints(img)
    if keypoints is not None:
        data.append(keypoints)
        labels.append(1)  # Label '1' for the target pose
        for _ in range(10):  # Generate 10 augmentations per image
            augmented_img = augment_image(img)
            keypoints_augmented = extract_keypoints(augmented_img)
            if keypoints_augmented is not None:
                data.append(keypoints_augmented)
                labels.append(1)

# Process non-pose images
for img in non_pose_images:
    keypoints = extract_keypoints(img)
    if keypoints is not None:
        data.append(keypoints)
        labels.append(0)  # Label '0' for non-pose images
        for _ in range(10):  # Generate 10 augmentations per image
            augmented_img = augment_image(img)
            keypoints_augmented = extract_keypoints(augmented_img)
            if keypoints_augmented is not None:
                data.append(keypoints_augmented)
                labels.append(0)

# Convert to NumPy arrays for training
data = np.array(data)
labels = np.array(labels)

# Train KNN on pose keypoints
knn.fit(data, labels)

# Function to evaluate a new image's pose accuracy and overlay confidence on the image
def evaluate_pose(image):
    if image is None:
        print("Error: Image not loaded for evaluation.")
        return None
    keypoints = extract_keypoints(image)
    if keypoints is None:
        print("No pose detected in the image.")
        return None
    prediction = knn.predict([keypoints])[0]
    confidence = knn.predict_proba([keypoints])[0][prediction]
    
    # Overlay confidence score on the image
    text = f"Confidence: {confidence * 100:.2f}%"
    color = (0, 255, 0) if prediction == 1 else (0, 0, 255)
    cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    
    return prediction, confidence, image

# Test with a new image
test_image = cv2.imread('PoseData/4.png')  # Replace with the path to your test image
if test_image is not None:
    result, confidence, output_image = evaluate_pose(test_image)
    if result == 1:
        print(f"Pose match with confidence: {confidence * 100:.2f}%")
    else:
        print(f"No match for the pose. Confidence in non-match: {confidence * 100:.2f}%")
    
    # Display the image
    cv2.imshow("Pose Evaluation", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Optional: Save the image with the overlay
    cv2.imwrite('pose_evaluation_result.png', output_image)
else:
    print("Error: Test image not loaded. Check the file path.")


C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


No match for the pose. Confidence in non-match: 66.67%


In [16]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Function to extract keypoints from an image
def extract_keypoints(image):
    if image is None:
        print("Warning: Empty image passed to extract_keypoints function.")
        return None
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.pose_landmarks:
        return None
    keypoints = []
    for landmark in results.pose_landmarks.landmark:
        keypoints.append([landmark.x, landmark.y])
    return np.array(keypoints)

# Function to calculate angles between landmarks
def calculate_angle(a, b, c):
    """Calculates the angle between three points (a, b, c)."""
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (vertex)
    c = np.array(c)  # Third point
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Extract features (angles) from an image
def extract_angles(image):
    keypoints = extract_keypoints(image)
    if keypoints is None:
        return None
    # Define keypoints to calculate angles (customize based on your use case)
    angles = [
        calculate_angle(keypoints[11], keypoints[13], keypoints[15]),  # Left elbow
        calculate_angle(keypoints[12], keypoints[14], keypoints[16]),  # Right elbow
        calculate_angle(keypoints[13], keypoints[11], keypoints[23]),  # Left shoulder
        calculate_angle(keypoints[14], keypoints[12], keypoints[24]),  # Right shoulder
        calculate_angle(keypoints[23], keypoints[25], keypoints[27]),  # Left hip
        calculate_angle(keypoints[24], keypoints[26], keypoints[28]),  # Right hip
    ]
    return np.array(angles)

# Load pose and non-pose images
pose_images = [cv2.imread(f'PoseData/{i}.png') for i in range(1, 24)]  # Replace with actual paths
non_pose_images = [cv2.imread(f'PoseData/f{i}.png') for i in range(1, 9)]  # Replace with actual paths

data, labels = [], []

# Process pose images
for img in pose_images:
    angles = extract_angles(img)
    if angles is not None:
        data.append(angles)
        labels.append(1)  # Label '1' for pose

# Process non-pose images
for img in non_pose_images:
    angles = extract_angles(img)
    if angles is not None:
        data.append(angles)
        labels.append(0)  # Label '0' for non-pose

# Convert data to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define a simple neural network for classification
model = Sequential([
    Dense(64, activation='relu', input_shape=(data.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Output a single confidence score
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_test, y_test))

# Function to evaluate a new image
def evaluate_pose(image, threshold=0.5):
    angles = extract_angles(image)
    if angles is None:
        print("No pose detected in the image.")
        return None
    angles = angles.reshape(1, -1)  # Reshape for prediction
    confidence = model.predict(angles)[0][0]  # Get confidence score
    match = confidence > threshold  # Check if above threshold
    
    # Overlay results on image
    text = f"Confidence: {confidence * 100:.2f}%, Match: {'Yes' if match else 'No'}"
    color = (0, 255, 0) if match else (0, 0, 255)
    cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    
    return confidence, match, image

# Test with a new image
import os
def evaluate_folder(folder_path, threshold=0.5, output_folder="output"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        image = cv2.imread(file_path)
        if image is None:
            print(f"Error: Unable to load image {filename}.")
            continue
        
        angles = extract_angles(image)
        if angles is None:
            print(f"No pose detected in {filename}.")
            continue
        
        # Predict pose confidence
        angles = angles.reshape(1, -1)  # Reshape for prediction
        confidence = model.predict(angles)[0][0]  # Get confidence score
        match = confidence > threshold  # Check if above threshold
        
        # Overlay results on image
        text = f"Confidence: {confidence * 100:.2f}%, Match: {'Yes' if match else 'No'}"
        color = (0, 255, 0) if match else (0, 0, 255)
        cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        
        # Save the result
        output_path = os.path.join(output_folder, f"result_{filename}")
        cv2.imwrite(output_path, image)
        print(f"Processed {filename}: {text}")

# Evaluate all images in a folder
evaluate_folder("PoseData")


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.3994 - loss: 21.3396 - val_accuracy: 0.5714 - val_loss: 3.8212
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5488 - loss: 11.4365 - val_accuracy: 0.5714 - val_loss: 2.2414
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4089 - loss: 15.0142 - val_accuracy: 0.5714 - val_loss: 5.2172
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5560 - loss: 8.9122 - val_accuracy: 0.5714 - val_loss: 8.2214
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7440 - loss: 1.7951 - val_accuracy: 0.5714 - val_loss: 10.6167
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6405 - loss: 7.8067 - val_accuracy: 0.5714 - val_loss: 11.9381
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9113 - loss: 3.6950 - val_accuracy: 0.5714 - val_loss: 11.7983
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6565 - loss: 5.5746 - val_accuracy: 0.5714 - val_loss: 1

In [17]:
import cv2
import mediapipe as mp
import os
import pandas as pd

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Function to extract landmarks from an image
def extract_landmarks(image):
    if image is None:
        print("Warning: Empty image passed to extract_landmarks function.")
        return None
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.pose_landmarks:
        return None
    # Extract (x, y, z) coordinates of all landmarks
    landmarks = []
    for landmark in results.pose_landmarks.landmark:
        landmarks.extend([landmark.x, landmark.y, landmark.z])  # Flatten as [x1, y1, z1, x2, y2, z2, ...]
    return landmarks

# Function to process a folder of images and save landmarks to a CSV
def save_landmarks_to_csv(folder_path, output_csv="landmarks.csv"):
    data = []
    filenames = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        image = cv2.imread(file_path)
        if image is None:
            print(f"Error: Unable to load image {filename}.")
            continue
        
        landmarks = extract_landmarks(image)
        if landmarks is None:
            print(f"No landmarks detected in {filename}.")
            continue
        
        filenames.append(filename)
        data.append(landmarks)
    
    # Create a DataFrame
    columns = []
    for i in range(33):  # 33 landmarks, each with x, y, z
        columns.extend([f"Landmark_{i}_x", f"Landmark_{i}_y", f"Landmark_{i}_z"])
    df = pd.DataFrame(data, columns=columns)
    df.insert(0, "Filename", filenames)  # Add filenames as the first column

    # Save to CSV
    df.to_csv(output_csv, index=False)
    print(f"Landmarks saved to {output_csv}")

# Process the folder and save landmarks
save_landmarks_to_csv("PoseData")  # Replace "TestData" with the path to your folder of images


Error: Unable to load image .ipynb_checkpoints.
Landmarks saved to landmarks.csv
